In [12]:
import tensorflow as tf
faggot = None
tf.reset_default_graph()
file = '/home/brandon/terabyte/Datasets/test_data/train_from.txt.ids1000'
inp_prod = tf.train.string_input_producer([file])
reader = tf.TextLineReader()
keyfuck, valfuck = reader.read(inp_prod)
batch_fuck_penis, nigger = tf.contrib.training.bucket_by_sequence_length(
    2,
    [valfuck], 
    4, 
    [2, 5], 
    dynamic_pad=True,
    name='titbals', 
)
with tf.Session() as stfu:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #print(tf.compat.as_str(stfu.run(valfuck)))
    #print(tf.compat.as_str(stfu.run(valfuck)))
    print(stfu.run([batch_fuck_penis, nigger]))
    print(stfu.run([batch_fuck_penis, nigger]))
    print(stfu.run([batch_fuck_penis, nigger]))
    print(stfu.run([batch_fuck_penis, nigger]))
    coord.request_stop()
    coord.join(threads)

[array([2, 2, 2, 2], dtype=int32), [array([b'29 5', b'11 7 9 27 4', b'17 8 22 43 28 4',
       b'10 7 6 14 23 6 31 19 4'], dtype=object)]]
[array([2, 2, 2, 2], dtype=int32), [array([b'10 7 9 15 14 4', b'42 32 37 5', b'13 8 12 4', b'40 44 9 15 45 5'], dtype=object)]]
[array([2, 2, 2, 2], dtype=int32), [array([b'13 30 26 12 35 41 6 39 4', b'11 20 8 24 16 4',
       b'36 47 38 25 33 5', b'34 46 18 21 4'], dtype=object)]]
[array([2, 2, 2, 2], dtype=int32), [array([b'10 49 18 52 4', b'17 48 16 50 5', b'17 48 16 51 5', b'29 5'], dtype=object)]]


In [35]:
batch_fuck_penis, nigger = tf.contrib.training.bucket_by_sequence_length(
    20,
    [valfuck], 
    2, 
    [2, 5], 
    name='titbals', 
    dyna
)

#nigger_params = tf.get_variable('fuck_you_faggot', [154, 666])
#$nigger_embedded = tf.nn.embedding_lookup(nigger_params, nigger, name="eatshitanddie")

SyntaxError: positional argument follows keyword argument (<ipython-input-35-691236e52f94>, line 7)

omg this works

In [15]:
with tf.Session() as faggot:
    faggot.run(tf.global_variables_initializer())
    # Start populating the filename queue
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for piece_of_shit in range(5):
        suckmydick = faggot.run(nigger)
        print(suckmydick)
        print(type(suckmydick))
        print(suckmydick.shape)
        print(suckmydick.dtype)
        print(suckmydick[0])
        print(type(suckmydick[0]))
    coord.request_stop()
    coord.join(threads)
    
print('lasdfa')
faggot = None
tf.reset_default_graph()

[b'29 5' b'11 7 9 27 4']
<class 'numpy.ndarray'>
(2,)
object
b'29 5'
<class 'bytes'>
[b'17 8 22 43 28 4' b'10 7 6 14 23 6 31 19 4']
<class 'numpy.ndarray'>
(2,)
object
b'17 8 22 43 28 4'
<class 'bytes'>
[b'10 7 9 15 14 4' b'42 32 37 5']
<class 'numpy.ndarray'>
(2,)
object
b'10 7 9 15 14 4'
<class 'bytes'>
[b'13 8 12 4' b'40 44 9 15 45 5']
<class 'numpy.ndarray'>
(2,)
object
b'13 8 12 4'
<class 'bytes'>
[b'13 30 26 12 35 41 6 39 4' b'11 20 8 24 16 4']
<class 'numpy.ndarray'>
(2,)
object
b'13 30 26 12 35 41 6 39 4'
<class 'bytes'>
lasdfa


In [ ]:
        print(type(batch))
        print(type(batch[0]))
        
    coord.request_stop()
    coord.join(threads)
faggot = None
tf.reset_default_graph()

In [ ]:
tf.contrib.training.bucket_by_sequence_length??

In [ ]:
fuck

In [ ]:
sess.run(fuck)

In [ ]:
print('fag')